In [1]:
path_dataset ='/home/marrakchi/TFM/TFM_env/Datasets/ExKaldiASRDatasetFormat/' 

In [2]:
# Systems Libraries
import os
import time

# For audio processing
import librosa
import librosa.display
import IPython as ipd
from IPython.display import display, HTML # For displaying tables

# For data processing
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# For visualization
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# For text processing
import string

from datetime import datetime

# Import exkaldi package
import exkaldi
exkaldi.info.reset_kaldi_root('/kaldi')

exkaldi.info.reset_kaldi_root( yourPath )
If not, ERROR will occur when implementing some core functions.


In [3]:
# ExKaldi Configuration
ExKaldiInfo = exkaldi.info
print(ExKaldiInfo)
print(ExKaldiInfo.KALDI)
print(ExKaldiInfo.KALDI_ROOT)

ExKaldiInfo(version='1.3.5.4', major='1', minor='3', patch='5', upload='4')
Kaldi(version='5.5', major='5', minor='5')
/kaldi


In [4]:
# Ensuring that GPU are detected/available and ready to use
print(torch.cuda.device_count())
print(torch.cuda.is_available())

1
True


In [5]:
# Load lexicon
lexicons = exkaldi.load_lex(os.path.join(path_dataset, "dict", "lexicons.lex"))
print(lexicons)
# Load transcriptions
trans = exkaldi.load_transcription(os.path.join(path_dataset,"text"))
trainTrans = exkaldi.load_transcription(os.path.join(path_dataset,'data/train',"text"))
valTrans = exkaldi.load_transcription(os.path.join(path_dataset,'data/val',"text"))
testTrans = exkaldi.load_transcription(os.path.join(path_dataset,'data/test',"text"))
# Load wavScp
wavScp = exkaldi.load_list_table(os.path.join(path_dataset,"wav.scp") )
wavScp_train = exkaldi.load_list_table(os.path.join(path_dataset,'data/train',"wav.scp") )
wavScp_val = exkaldi.load_list_table(os.path.join(path_dataset,'data/val',"wav.scp") )
wavScp_test = exkaldi.load_list_table(os.path.join(path_dataset,'data/test',"wav.scp") )

# Training language model

In [78]:
## Train N-Grams language model
# 1. Train a LM.
# We have trained 2,3,4 grams model with both srilm and kenlm and chose the best one, which is 3-grams model back kenlm.
# So we directly train this one.
exkaldi.lm.train_ngrams_kenlm(
                        lexicons,
                        order=3,
                        text=trainTrans,  # (exkaldi Transcription object)--> the information of utterance IDs will be omitted automatically.
                        outFile=os.path.join(path_dataset,"lm",f"3grams.arpa"), 
                        config={"--discount_fallback":True,"-S":"20%"},
                    )

print(f"Generate ARPA language model done.")

# First error --> permission denied 
#    chmod 0775 /home/marrakchi/TFM/TFM_env/exkaldisrc/tools/lmplz
# Second error --> error while loading shared libraries: libboost_program_options.so.1.65.1
#    SOLUTION --> sudo apt-get install libboost-program-options-dev

# ARPA model can be transform to binary format in order to accelerate loading and reduce memory cost.
exkaldi.lm.arpa_to_binary(
                            arpaFile=os.path.join(path_dataset,"lm",f"3grams.arpa"),
                            outFile=os.path.join(path_dataset,"lm",f"3grams.binary"),
                        )
# First error --> permission denied 
#    chmod 0775 /home/marrakchi/TFM/TFM_env/exkaldisrc/tools/build_binary

# Train 2-gram KENLM model
exkaldi.lm.train_ngrams_kenlm(
                        lexicons,
                        order=2,
                        text=trainTrans,  # (exkaldi Transcription object)--> the information of utterance IDs will be omitted automatically.
                        outFile=os.path.join(path_dataset,"lm",f"2grams.arpa"), 
                        config={"--discount_fallback":True,"-S":"20%"},
                    )
exkaldi.lm.arpa_to_binary(
                            arpaFile=os.path.join(path_dataset,"lm",f"2grams.arpa"),
                            outFile=os.path.join(path_dataset,"lm",f"2grams.binary"),
                        )

# Train 4-gram KENLM model
exkaldi.lm.train_ngrams_kenlm(
                        lexicons,
                        order=4,
                        text=trainTrans,  # (exkaldi Transcription object)--> the information of utterance IDs will be omitted automatically.
                        outFile=os.path.join(path_dataset,"lm",f"4grams.arpa"), 
                        config={"--discount_fallback":True,"-S":"20%"},
                    )
exkaldi.lm.arpa_to_binary(
                            arpaFile=os.path.join(path_dataset,"lm",f"4grams.arpa"),
                            outFile=os.path.join(path_dataset,"lm",f"4grams.binary"),
                        )

Generate ARPA language model done.


In [10]:
# Train 5-gram KENLM model
exkaldi.lm.train_ngrams_kenlm(
                        lexicons,
                        order=5,
                        text=trainTrans,  # (exkaldi Transcription object)--> the information of utterance IDs will be omitted automatically.
                        outFile=os.path.join(path_dataset,"lm",f"5grams.arpa"), 
                        config={"--discount_fallback":True,"-S":"20%"},
                    )
exkaldi.lm.arpa_to_binary(
                            arpaFile=os.path.join(path_dataset,"lm",f"5grams.arpa"),
                            outFile=os.path.join(path_dataset,"lm",f"5grams.binary"),
                        )

# Train 6-gram KENLM model
exkaldi.lm.train_ngrams_kenlm(
                        lexicons,
                        order=6,
                        text=trainTrans,  # (exkaldi Transcription object)--> the information of utterance IDs will be omitted automatically.
                        outFile=os.path.join(path_dataset,"lm",f"6grams.arpa"), 
                        config={"--discount_fallback":True,"-S":"20%"},
                    )
exkaldi.lm.arpa_to_binary(
                            arpaFile=os.path.join(path_dataset,"lm",f"6grams.arpa"),
                            outFile=os.path.join(path_dataset,"lm",f"6grams.binary"),
                        )

'/home/marrakchi/TFM/TFM_env/Datasets/ExKaldiASRDatasetFormat/lm/6grams.binary'

In [12]:
# 2. Test the model by computing the perplexity.

# initialize a Python KenLM n-grams object
print('\n----------------------------------3-GRAMS--------------------------------------------------------------')
model = exkaldi.load_ngrams( os.path.join(path_dataset,"lm",f"3grams.binary") )
print(model.info)
perScore = model.perplexity(testTrans)
print(f"\nThe weighted average perplexity of this model is: {perScore}.")
# 3. score
score = model.score(testTrans)
print('\nMean Score --> ', score.mean())
print('\nMean Score by length of sentence-->', score.mean( weight= testTrans.sentence_length() ))
print('\n----------------------------------2-GRAMS--------------------------------------------------------------')
model = exkaldi.load_ngrams( os.path.join(path_dataset,"lm",f"2grams.binary") )
print(model.info)
perScore = model.perplexity(testTrans)
score = model.score(testTrans)
print(f"\nThe weighted average perplexity of this model is: {perScore}.")
print('\nMean Score --> ', score.mean())
print('\nMean Score by length of sentence-->', score.mean( weight= testTrans.sentence_length() ))
print('\n----------------------------------4-GRAMS--------------------------------------------------------------')
model = exkaldi.load_ngrams( os.path.join(path_dataset,"lm",f"4grams.binary") )
print(model.info)
perScore = model.perplexity(testTrans)
score = model.score(testTrans)
print(f"\nThe weighted average perplexity of this model is: {perScore}.")
print('\nMean Score --> ', score.mean())
print('\nMean Score by length of sentence-->', score.mean( weight= testTrans.sentence_length() ))


----------------------------------3-GRAMS--------------------------------------------------------------
NgramInfo(order=3, path="b'/home/marrakchi/TFM/TFM_env/Datasets/ExKaldiASRDatasetFormat/lm/3grams.binary'")

The weighted average perplexity of this model is: PPL(prob=-817979.52, sentences=24756, words=291975, ppl=382.44, ppl1=633.2).

Mean Score -->  -33.04166745975804

Mean Score by length of sentence--> -41.17497238319086

----------------------------------2-GRAMS--------------------------------------------------------------
NgramInfo(order=2, path="b'/home/marrakchi/TFM/TFM_env/Datasets/ExKaldiASRDatasetFormat/lm/2grams.binary'")

The weighted average perplexity of this model is: PPL(prob=-839317.55, sentences=24756, words=291975, ppl=446.62, ppl1=749.24).

Mean Score -->  -33.90360111887071

Mean Score by length of sentence--> -42.37599998133139

----------------------------------4-GRAMS--------------------------------------------------------------
NgramInfo(order=4, path="b'/

In [11]:
print('\n----------------------------------5-GRAMS--------------------------------------------------------------')
model = exkaldi.load_ngrams( os.path.join(path_dataset,"lm",f"5grams.binary") )
print(model.info)
perScore = model.perplexity(testTrans)
print(f"\nThe weighted average perplexity of this model is: {perScore}.")
# 3. score
score = model.score(testTrans)
print('\nMean Score --> ', score.mean())
print('\nMean Score by length of sentence-->', score.mean( weight= testTrans.sentence_length() ))
print('\n----------------------------------6-GRAMS--------------------------------------------------------------')
model = exkaldi.load_ngrams( os.path.join(path_dataset,"lm",f"6grams.binary") )
print(model.info)
perScore = model.perplexity(testTrans)
score = model.score(testTrans)
print(f"\nThe weighted average perplexity of this model is: {perScore}.")
print('\nMean Score --> ', score.mean())
print('\nMean Score by length of sentence-->', score.mean( weight= testTrans.sentence_length() ))


----------------------------------5-GRAMS--------------------------------------------------------------
NgramInfo(order=5, path="b'/home/marrakchi/TFM/TFM_env/Datasets/ExKaldiASRDatasetFormat/lm/5grams.binary'")

The weighted average perplexity of this model is: PPL(prob=-810374.96, sentences=24756, words=291975, ppl=361.88, ppl1=596.34).

Mean Score -->  -32.73448707470314

Mean Score by length of sentence--> -40.711886746512704

----------------------------------6-GRAMS--------------------------------------------------------------
NgramInfo(order=6, path="b'/home/marrakchi/TFM/TFM_env/Datasets/ExKaldiASRDatasetFormat/lm/6grams.binary'")

The weighted average perplexity of this model is: PPL(prob=-810391.23, sentences=24756, words=291975, ppl=361.92, ppl1=596.42).

Mean Score -->  -32.73514416969387

Mean Score by length of sentence--> -40.711357113193095


In [12]:
# ------- Make Grammar fst ------
exkaldi.decode.graph.make_G(
                        lexicons, 
                        arpaFile=os.path.join(path_dataset,"lm",f"5grams.arpa"),
                        outFile=os.path.join(path_dataset,"lm",f"G.5.fst"), 
                        order=4
                    )
print(f"Make Grammar fst done.")

# ------- Compose LG fst for futher use ------
exkaldi.decode.graph.compose_LG(
                        LFile=os.path.join(path_dataset,"dict","L_disambig.fst"), 
                        GFile=os.path.join(path_dataset,"lm",f"G.5.fst"),
                        outFile=os.path.join(path_dataset,"lm",f"LG.5.fst"),
                    )
print(f"Compose LG fst done.")

Make Grammar fst done.
Compose LG fst done.


In [ ]:
"""
# Train 4-gram SRILM model
exkaldi.lm.train_ngrams_srilm(
                        lexicons,
                        order=4,
                        text=trainTrans,  # (exkaldi Transcription object)--> the information of utterance IDs will be omitted automatically.
                        outFile=os.path.join(path_dataset,"lm",f"4grams_srilm.arpa"), 
                        config={"--discount_fallback":True,"-S":"20%"},
                    )
exkaldi.lm.arpa_to_binary(
                            arpaFile=os.path.join(path_dataset,"lm",f"4grams_srilm.arpa"),
                            outFile=os.path.join(path_dataset,"lm",f"4grams_srilm.binary"),
                        )
model = exkaldi.load_ngrams( os.path.join(path_dataset,"lm",f"4grams_srilm.binary") )
print(model.info)
perScore = model.perplexity(testTrans)
score = model.score(testTrans)
print(f"\nThe weighted average perplexity of this model is: {perScore}.")
print('\nMean Score --> ', score.mean())
print('\nMean Score by length of sentence-->', score.mean( weight= testTrans.sentence_length() ))
"""